In [17]:
!pip install mlxtend

In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# numpy
import numpy as np # linear algebra
print('numpy: {}'.format(np.__version__))
# pandas
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
print('pandas: {}'.format(pd.__version__))
import math
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
import matplotlib.pyplot as plt
# scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))
import seaborn as sns
print('seaborn: {}'.format(sns.__version__))
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

numpy: 1.23.5
pandas: 1.5.3
Python: 3.10.10 | packaged by conda-forge | (main, Mar 24 2023, 20:08:06) [GCC 11.3.0]
scipy: 1.10.1
matplotlib: 3.7.1
sklearn: 1.2.2
seaborn: 0.12.2
/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


# * In order to  avoid problems with missing values I dropped the cases containning N/A values *

In [19]:
fpath='/kaggle/input/icr-identify-age-related-conditions/'
train= pd.read_csv(fpath + 'train.csv', low_memory=False)

df2 = pd.DataFrame(train)
df  = df2.dropna(how='any',axis =0)
df  = df.drop('EJ', axis = 1)
df  = df.drop('Id', axis = 1)
#print (df)

# It is the same using: df=df1.dropna(how='any',axis =1)
#display(df)

In [20]:
#the_pairs =
#ns.pairplot(assays)
#plt.show()
#plt.savefig('correlat.png')

In [21]:
# I will get the '1' and '0' classifieds apparts

from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

#print(df.dtypes)

df['Class'] = df['Class'].astype('bool')
positives = df.loc[df['Class'] == True]
negatives =  df.loc[df['Class'] == False]

posClass = positives['Class']
#posClass = posClass.astype('bool')
negClass = negatives['Class']
#negClass = negClass.astype('bool')

positives = positives.drop('Class', axis = 1)
#print ('Printing matrix Positive Cases ---------------------------------------\n', positives)

negatives = negatives.drop('Class', axis = 1)
#print ('Printing matrix Negative Cases---------------------------------------\n', negatives)

# Split dataset into training set and test set
Xp_train, Xp_test, yp_train, yp_test = train_test_split(positives, posClass, test_size=0.3, random_state=1) # 70% training and 30% test
Xn_train, Xn_test, yn_train, yn_test = train_test_split(negatives, negClass, test_size=0.3, random_state=1) # 70% training and 30% test

X_train = pd.concat([Xp_train, Xn_train], ignore_index=True)
y_train = pd.concat([yp_train, yn_train], ignore_index=True)

X_test = pd.concat([Xp_test, Xn_test], ignore_index=True)
y_test = pd.concat([yp_test, yn_test], ignore_index=True)

assays = df.drop ('Class', axis = 1)
#print('Printing train Xmatrix ---------------------------------------\n',X_train)

#print('Printing test matrix ---------------------------------------\n',X_test)


In [22]:

# Create Decision Tree classifer object
clf = DecisionTreeClassifier(criterion = 'entropy', max_depth= 8)

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
yr_pred = clf.predict(X_test)

# Model Accuracy, how often is the classifier correct?

print("-------------- Accuracy:",metrics.accuracy_score(y_test, yr_pred), "-------------- \n")


# Here we predict the whole dataset
validar = clf.predict(assays)
#display (valida)
print("\n\n-------------- Overall Accuracy:",metrics.accuracy_score(validar, df['Class']), "-------------- \n")

-------------- Accuracy: 0.8424242424242424 -------------- 




-------------- Overall Accuracy: 0.9507299270072993 -------------- 



In [28]:
#!conda config --add channels conda-forge
!conda install pydotplus
#!pip install six

import graphviz
from sklearn.tree import export_graphviz
from six import StringIO  
from IPython.display import Image  
import pydotplus


CondaHTTPError: HTTP 000 CONNECTION FAILED for url <https://conda.anaconda.org/conda-forge/linux-64/current_repodata.json>
Elapsed: -

An HTTP error occurred when trying to retrieve this URL.
HTTP errors are often intermittent, and a simple retry will get you on your way.
'https//conda.anaconda.org/conda-forge/linux-64'




ModuleNotFoundError: No module named 'pydotplus'

In [ ]:
print ('\n\n -------------------------------------------- Results from Training Data ---------------------------------------------------------\n\n')

'''feature_names = feature_cols, '''
dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True, feature_names= list(Xr_train.columns.values),
                special_characters=True , class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('/kaggle/working/ICR-treeTrain.png')
Image(graph.create_png())

## *Stepwise Regression*

In [ ]:

from sklearn import linear_model as lm
from sklearn.metrics import accuracy_score
from mlxtend.feature_selection import SequentialFeatureSelector
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

sfs = SequentialFeatureSelector(lm.LogisticRegression(max_iter = 10000),
                                k_features=6,
                                forward=True,
                                scoring='accuracy',
                                cv=None)
selected_features = sfs.fit(X_train,y_train.T)#

fig1 = plot_sfs(sfs.get_metric_dict(), kind='std_dev')

plt.ylim([0.8, 1])
plt.title('Sequential Forward Selection (w. StdDev)')
plt.grid()
plt.show()

print ("Subsets",sfs.subsets_)
print ("\n********************\n -rsquare : ",sfs.k_score_)


In [ ]:
Xr_train =  X_train[['AR', 'BC', 'CW ', 'DA', 'DU', 'FI']].copy()
assaysR =  assays[['AR', 'BC', 'CW ', 'DA', 'DU', 'FI']].copy()
Xr_test =  X_test[['AR', 'BC', 'CW ', 'DA', 'DU', 'FI']].copy()

In [ ]:

# Create Decision Tree classifer object
clf = DecisionTreeClassifier(criterion = 'entropy', max_depth= 8)

# Train Decision Tree Classifer
clf = clf.fit(Xr_train,y_train)

#Predict the response for test dataset
yr_pred = clf.predict(Xr_test)

# Model Accuracy, how often is the classifier correct?

print("-------------- Accuracy:",metrics.accuracy_score(y_test, yr_pred), "-------------- \n")


# Here we predict the whole dataset
validar = clf.predict(assaysR)
#display (valida)
print("\n\n-------------- Overall Accuracy:",metrics.accuracy_score(validar, df['Class']), "-------------- \n")

In [ ]:
print ('\n\n -------------------------------------------- Results from All Data ---------------------------------------------------------\n\n')

'''feature_names = feature_cols, '''
dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True, feature_names= list(Xr_train.columns.values),
                special_characters=True , class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('ICR-treeTrain_reduced.png')
Image(graph.create_png())

### *After reducing the variables as suggested by Stepwise, the accuracy is worse (reduced variables: 0.855 original variables: 0.885) than with the full set, the same holds for the overall Accuracy (0.947 vs 0.9635). This can be circunvected at real case iterating and saving the models in order to use the best one at production time.*

In [ ]:
fpath='/kaggle/input/icr-identify-age-related-conditions/'
test = pd.read_csv(fpath + 'test.csv', low_memory=False)

df3 = pd.DataFrame(test)
X_submit= df3.drop["Id"]

# Create Decision Tree classifer object
clf = DecisionTreeClassifier(criterion = 'entropy', max_depth= 8)

# Train Decision Tree Classifer
clf = clf.fit(assays,df['Class'])

#Predict the response for test dataset
y_submit = clf.predict(X_submit)

# Model Accuracy, how often is the classifier correct?

print("-------------- Accuracy:",metrics.accuracy_score(y_test, yr_pred), "-------------- \n")



In [ ]:
f_out='/kaggle/working'
import csv
'''
header = ['Id', 'class_0', 'class_1']

with open('countries.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)
    for y in y_submit:
       data = [df3['Id'], y, 1 - y] 
       # write the data
       writer.writerow(data)
'''

In [ ]:
print ('\n\n -------------------------------------------- Results from Training Data ---------------------------------------------------------\n\n')

'''feature_names = feature_cols, '''
dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True, feature_names= list(Xr_train.columns.values),
                special_characters=True , class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('/kaggle/working/ICR-treeSubmit.png')
Image(graph.create_png())